In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['RPE']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [25]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [26]:
list(zip(longforms, counts))

[('retinal pigment epithelium', 2490),
 ('retinal pigment epithelial', 1118),
 ('rating of perceived exertion', 152),
 ('retinal pigment epithelial cells', 108),
 ('reward prediction error', 63),
 ('retinal pigment epithelia', 18),
 ('retina pigment epithelium', 16),
 ('retinal pigment epithelium cells', 7),
 ('rapid palatal expansion', 6),
 ('related proliferative effects', 6),
 ('retina pigment epithelial', 6),
 ('retinal pigmentary epithelium', 5),
 ('rat placenta extract', 5),
 ('rice prolamin extract', 5),
 ('r phycoerythrin', 5),
 ('retinal pigment endothelial', 4),
 ('respiratory protective equipment', 3),
 ('re expansion pulmonary edema', 3),
 ('refractory partial epilepsy', 2),
 ('related packaging efficiently', 2),
 ('related predispositional effects', 2),
 ('radix puerariae extract', 2),
 ('rat pancreatic extract', 2),
 ('rat prostatic extract', 2),
 ('retinal pigment', 2),
 ('radical prostatectomy', 2),
 ('retinal pigment epithelial layer', 2),
 ('retinal pigment epithelium

In [30]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [31]:
result = [grounding_map, names, pos_labels]

In [32]:
result

[{'positive prediction error': 'ungrounded',
  'r phycoerythrin': 'MESH:D010799',
  'radical prostatectomy': 'ungrounded',
  'radix puerariae extract': 'ungrounded',
  'rapid palatal expansion': 'ungrounded',
  'rat pancreatic extract': 'ungrounded',
  'rat placenta extract': 'ungrounded',
  'rat prostatic extract': 'ungrounded',
  'rating and perceptual': 'ungrounded',
  'rating of perceived exertion': 'NCIT:C122028',
  'rating of perceived exertion scale': 'NCIT:C122028',
  're expansion pulmonary edema': 'MESH:D011654',
  'refractory partial epilepsy': 'ungrounded',
  'related packaging efficiently': 'ungrounded',
  'related predispositional effects': 'ungrounded',
  'related proliferative effects': 'ungrounded',
  'respiratory protective equipment': 'MESH:D012134',
  'retina pigment epithelial': 'MESH:D055213',
  'retina pigment epithelium': 'MESH:D055213',
  'retinal pigment': 'MESH:D055213',
  'retinal pigment endothelial': 'MESH:D055213',
  'retinal pigment epithelia': 'MESH:D05

In [33]:
grounding_map, names, pos_labels = [{'positive prediction error': 'ungrounded',
  'r phycoerythrin': 'MESH:D010799',
  'radical prostatectomy': 'ungrounded',
  'radix puerariae extract': 'ungrounded',
  'rapid palatal expansion': 'ungrounded',
  'rat pancreatic extract': 'ungrounded',
  'rat placenta extract': 'ungrounded',
  'rat prostatic extract': 'ungrounded',
  'rating and perceptual': 'ungrounded',
  'rating of perceived exertion': 'NCIT:C122028',
  'rating of perceived exertion scale': 'NCIT:C122028',
  're expansion pulmonary edema': 'MESH:D011654',
  'refractory partial epilepsy': 'ungrounded',
  'related packaging efficiently': 'ungrounded',
  'related predispositional effects': 'ungrounded',
  'related proliferative effects': 'ungrounded',
  'respiratory protective equipment': 'MESH:D012134',
  'retina pigment epithelial': 'MESH:D055213',
  'retina pigment epithelium': 'MESH:D055213',
  'retinal pigment': 'MESH:D055213',
  'retinal pigment endothelial': 'MESH:D055213',
  'retinal pigment epithelia': 'MESH:D055213',
  'retinal pigment epithelial': 'MESH:D055213',
  'retinal pigment epithelial cells': 'MESH:D055213',
  'retinal pigment epithelial layer': 'MESH:D055213',
  'retinal pigment epithelium': 'MESH:D055213',
  'retinal pigment epithelium 1': 'MESH:D055213',
  'retinal pigment epithelium cells': 'MESH:D055213',
  'retinal pigmentary epithelium': 'MESH:D055213',
  'reward prediction error': 'reward_prediction_error',
  'ribulose 5 phosphate 3 epimerase': 'HGNC:10293',
  'ribulose phosphate 3 epimerase': 'HGNC:10293',
  'rice prolamin extract': 'ungrounded',
  'subretinal pigment epithelium': 'MESH:D055213'},
 {'MESH:D010799': 'Phycoerythrin',
  'NCIT:C122028': 'Rating of Perceived Exertion',
  'MESH:D011654': 'Pulmonary Edema',
  'MESH:D012134': 'Respiratory Protective Devices',
  'MESH:D055213': 'Retinal Pigment Epithelium',
  'reward_prediction_error': 'reward_prediction_error',
  'HGNC:10293': 'RPE'},
 ['HGNC:10293', 'MESH:D055213', 'NCIT:C122028']]

In [34]:
excluded_longforms = []

In [35]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [36]:
additional_entities = {'HGNC:10293': ['RPE', ['RPE', 'ribulose-5-phosphate-3-epimerase']]}

In [37]:
unambiguous_agent_texts = {}

In [38]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [39]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [40]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [41]:
intersection1

[('HGNC:10293', 'HGNC:10293', 26)]

In [42]:
intersection2

[('MESH:D055213', 'HGNC:10293', 0),
 ('NCIT:C122028', 'HGNC:10293', 0),
 ('MESH:D011654', 'HGNC:10293', 0),
 ('ungrounded', 'HGNC:10293', 0),
 ('MESH:D010799', 'HGNC:10293', 0),
 ('HGNC:10293', 'HGNC:10293', 0),
 ('reward_prediction_error', 'HGNC:10293', 0),
 ('MESH:D012134', 'HGNC:10293', 0)]

In [43]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [44]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [45]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 03:48:48] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 03:49:37] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9908412380735351 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [46]:
classifier.stats

{'label_distribution': {'MESH:D055213': 2581,
  'NCIT:C122028': 107,
  'MESH:D011654': 2,
  'ungrounded': 22,
  'MESH:D010799': 3,
  'HGNC:10293': 7,
  'reward_prediction_error': 43,
  'MESH:D012134': 2},
 'f1': {'mean': 0.990841, 'std': 0.003162},
 'precision': {'mean': 0.985718, 'std': 0.004371},
 'recall': {'mean': 0.996287, 'std': 0.002038},
 'MESH:D055213': {'f1': {'mean': 0.993459, 'std': 0.001854},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.98701, 'std': 0.003656}},
 'MESH:D010799': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'reward_prediction_error': {'f1': {'mean': 0.964706, 'std': 0.028818},
  'pr': {'mean': 0.955556, 'std': 0.054433},
  'rc': {'mean': 0.977778, 'std': 0.044444}},
 'MESH:D012134': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'NCIT:C122028': {'f1': {'mean': 0.970642, 'std': 0.025445},
  'pr': {'mean': 0.952814, 'std': 0.052171}

In [47]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [48]:
disamb.dump(model_name, results_path)

In [49]:
print(disamb.info())

Disambiguation model for RPE

Produces the disambiguations:
	Phycoerythrin	MESH:D010799
	Pulmonary Edema	MESH:D011654
	RPE*	HGNC:10293
	Rating of Perceived Exertion*	NCIT:C122028
	Respiratory Protective Devices	MESH:D012134
	Retinal Pigment Epithelium*	MESH:D055213
	reward_prediction_error	reward_prediction_error

Class level metrics:
--------------------
Grounding                     	Count	F1     
    Retinal Pigment Epithelium*	2581	0.99346
  Rating of Perceived Exertion*	 107	0.97064
       reward_prediction_error	  43	0.96471
                    Ungrounded	  22	0.21333
                           RPE*	   7	0.33333
                 Phycoerythrin	   3	    0.0
               Pulmonary Edema	   2	    0.2
Respiratory Protective Devices	   2	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.99084
	Precision:	0.98572
	Recall:		0.99629

* Positive labels
See Docstring for explanation



In [50]:
model_to_s3(disamb)